In [14]:
%pip install chardet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Compilation of raw/2024 AND raw/2025 datasets

Note: Manually replace the year of the year to be processed

Run this for excel files that have invalid column names such as 0, 2, 3, 4, 5, 6, 7, 8, 9, 13

In [ ]:
import pandas as pd
from pathlib import Path
import os
import utilities as util

years = ['2024', '2025']

def discover_header(file_path, sheet_name='Member Consumers Data'):
    # Temporarily read with no header to inspect rows
    temp_df = pd.read_excel(file_path, sheet_name=sheet_name, header=None, usecols='A:N', engine='openpyxl')

    for i, row in temp_df.iterrows():
        normalized = [str(cell).strip().lower() for cell in row]
        if any('name' in cell for cell in normalized):
            print(f"✅ {file_path}: Header found at row {i}")
            return i

    print("❌ Header not found — fallback to default row")
    return Exception("Header not found")

# 🧹 Sanitize sheet reader
def read_sheet(file_path, is_first):
    try:
        header_row = discover_header(file_path)
        return pd.read_excel(
            file_path,
            sheet_name='Member Consumers Data',
            skiprows=header_row if is_first else 6,
            usecols='A:N',
            header=None if not is_first else 0,
            engine='openpyxl'
        )
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")
        return pd.DataFrame()

# 🚦 Main loop
output_folder = 'staging'
columns_to_exclude = ["Water Meter Serial #", "Meter Serial No."]

for year in years:
    base_dir = Path(f'../../dataset/raw/{year}/')
    for folder in base_dir.iterdir():
        os.makedirs(base_dir / output_folder, exist_ok=True)
        if not folder.is_dir():
            continue

        compiled_df = pd.DataFrame()
        try:
            for subfolder in folder.iterdir():
                if not subfolder.is_dir():
                    continue

                # Filter valid Excel files
                excel_files = [f for f in subfolder.iterdir()
                            if f.suffix in ('.xlsm', '.xlsx') and not f.name.startswith('~')]

                for i, file_path in enumerate(excel_files):
                    new_df = read_sheet(file_path, is_first=(i == 0))

                    # ✅ Validate new_df before appending
                    if not new_df.empty and new_df.dropna(axis=1, how='all').shape[1] > 0:
                        new_df.dropna(thresh=5, inplace=True, axis=0)
                        compiled_df = pd.concat([compiled_df, new_df], ignore_index=True)
                        compiled_df = compiled_df.loc[:, ~compiled_df.columns.isin(columns_to_exclude)].copy()
                        compiled_df.dropna(how='all', inplace=True, axis=1)

            # 📤 Export compiled data
            month = folder.name[0:3].upper()
            output_file = base_dir / output_folder / f'{month}_{year}.csv'
            util.save_csv(compiled_df, output_file)
            print(f"✅ Compiled: {output_file}")

        except Exception as e:
            print(f"Error accessing {folder.name}: {e}")

print("\n\n✅ All files compiled successfully!")

✅ ..\..\dataset\raw\2024\AUG_2024\HAZEL-Altarejos, Lique St\BILLING FORM_ALTAREJOS ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\HAZEL-Altarejos, Lique St\BILLING FORM_LIQUE ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\JINKY-Bartolabac, Grafilo, Villamor St\BILLING FORM_BARTOLABAC ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\JINKY-Bartolabac, Grafilo, Villamor St\BILLING FORM_GRAFILO ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\JINKY-Bartolabac, Grafilo, Villamor St\BILLING FORM_VILLAMOR ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\JOSHUA-Amican St., Burgos\BILLING FORM_AMICAN ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\JOSHUA-Amican St., Burgos\BILLING FORM_BINALIW, BURGOS.xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\JOSHUA-Amican St., Burgos\BILLING FORM_NHA DUPLEX.xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\AUG_2024\JOSHUA-Amican St.

C:\Users\Mark June Almojuela\AppData\Local\Temp\ipykernel_17824\757800937.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compiled_df = pd.concat([compiled_df, new_df], ignore_index=True)


✅ ..\..\dataset\raw\2024\SEP_2024\JINKY-Bartolabac, Grafilo, Villamor St\BILLING FORM_VILLAMOR ST..xlsm: Header found at row 5


C:\Users\Mark June Almojuela\AppData\Local\Temp\ipykernel_17824\757800937.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compiled_df = pd.concat([compiled_df, new_df], ignore_index=True)


✅ ..\..\dataset\raw\2024\SEP_2024\JOSHUA-Amican St., Burgos\BILLING FORM_AMICAN ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\JOSHUA-Amican St., Burgos\BILLING FORM_BINALIW, BURGOS.xlsm: Header found at row 5


C:\Users\Mark June Almojuela\AppData\Local\Temp\ipykernel_17824\757800937.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compiled_df = pd.concat([compiled_df, new_df], ignore_index=True)


✅ ..\..\dataset\raw\2024\SEP_2024\JOSHUA-Amican St., Burgos\BILLING FORM_NHA DUPLEX.xlsm: Header found at row 5


C:\Users\Mark June Almojuela\AppData\Local\Temp\ipykernel_17824\757800937.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compiled_df = pd.concat([compiled_df, new_df], ignore_index=True)


✅ ..\..\dataset\raw\2024\SEP_2024\JOSHUA-Amican St., Burgos\BILLING FORM_TALABA, BURGOS.xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\JOSHUA-Amican St., Burgos\BILLING FORM_TANGNAN, BURGOS.xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\JOSHUA-Amican St., Burgos\BILLING FORM_UPPER BLISS.xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\JUVY-Bailon, Balcavem, Jones St\BILLING FORM_BAILON ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\JUVY-Bailon, Balcavem, Jones St\BILLING FORM_BALCAVEM ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\JUVY-Bailon, Balcavem, Jones St\BILLING FORM_JONES ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\LUVILYN-Balintong, Don Juan, Flores, Moyot St\BILLING FORM_BALINTONG ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\2024\SEP_2024\LUVILYN-Balintong, Don Juan, Flores, Moyot St\BILLING FORM_DON JUAN REJUSO ST..xlsm: Header found at row 5
✅ ..\..\dataset\raw\202

Run this for excel files which are already compiled

In [ ]:
# Handle the conversion of xlsx to csv for datasets already in compiled form prior to preprocessing
import os 
from pathlib import Path
import pandas as pd
from utilities import save_csv

years = ['2019','2020', '2022', '2023']
output_folder = 'compiled'

def discover_header(file_path, sheet_name='Sheet1'):
    # Temporarily read with no header to inspect rows
    temp_df = pd.read_excel(file_path, sheet_name=sheet_name, header=None, usecols='A:H', engine='openpyxl')

    for i, row in temp_df.iterrows():
        normalized = [str(cell).strip().lower() for cell in row]
        if any('name' in cell for cell in normalized):
            print(f"✅ {file_path}: Header found at row {i}")
            return i

    print("❌ Header not found — fallback to default row")
    return Exception("Header not found")

for year in years:
    base_dir = Path(f'../../dataset/raw/{year}/')
    files = [f for f in base_dir.iterdir() if f.suffix in ('.xlsm', '.xlsx') and not f.name.startswith('~')]
    output_dir = base_dir / output_folder
    os.makedirs(output_dir, exist_ok=True)

    columns_to_exclude = ["Water Meter Serial #", "Meter Serial No."]

    for file in files:
        print(file.name)
        month = file.name[0:3].upper()

        header_row = discover_header(file)
        df = pd.read_excel(file, sheet_name = 'Sheet1', skiprows=header_row, usecols='A:H', header = 0, engine='openpyxl')
        final_df = df.loc[:, ~df.columns.isin(columns_to_exclude)].copy()
        final_df.dropna(how='all', inplace=True, axis=1)
        save_csv(final_df, output_dir / f'{month}_{year}.csv')

print("✅ All files processed successfully")

DECEMBER 10, 2019-JANUARY 10, 2020.xlsx
✅ ..\..\dataset\raw\2019\DECEMBER 10, 2019-JANUARY 10, 2020.xlsx: Header found at row 0
AUGUST 10-SEPTEMBER 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\AUGUST 10-SEPTEMBER 10, 2020.xlsx: Header found at row 2
FEBRUARY 10-MARCH 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\FEBRUARY 10-MARCH 10, 2020.xlsx: Header found at row 0
JANUARY 10-FEBRUARY 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\JANUARY 10-FEBRUARY 10, 2020.xlsx: Header found at row 0
JULY 10-AUGUST 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\JULY 10-AUGUST 10, 2020.xlsx: Header found at row 2
JUNE 10-JULY 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\JUNE 10-JULY 10, 2020.xlsx: Header found at row 2
MARCH 10-MAY 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\MARCH 10-MAY 10, 2020.xlsx: Header found at row 0
MAY 10-JUNE 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\MAY 10-JUNE 10, 2020.xlsx: Header found at row 0
NOVEMBER 10-DECEMBER 10, 2020.xlsx
✅ ..\..\dataset\raw\2020\NOVEMBER 10-DECEMBER 10, 2020.xlsx: Header found at row 0


In [ ]:
# Final dataset view in staging columnn
import utilities as utils
from pathlib import Path
import chardet

def determine_encoding(file_path):
    # Read first 100,000 bytes for detection
    with open(file_path, 'rb') as f:
        raw_data = f.read(100000)
    
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    confidence = result['confidence']
    
    print(f"Detected Encoding: {encoding} (confidence: {confidence})")
    return encoding

base_dir = Path(f'../../dataset/raw/')
# years = ['2024','2025']
years = ['2019', '2020', '2021', '2022', '2023']
# source_folder = 'staging'
source_folder = 'compiled'

for year in years:
    try:
        inner_path = base_dir / year / source_folder
        files = [f for f in inner_path.iterdir() if f.is_file() and not(f.name in ('staging', 'compiled') or f.name.startswith('COM') or f.name.startswith('STA'))]

        for file in files:
            print(file.name)
        file_path = base_dir / year / source_folder / file.name
        determine_encoding(file_path)
        df = utils.read_csv(file_path)
        print(df.info())
    except FileNotFoundError as e:
        print(e)

DEC_2019.csv
Detected Encoding: utf-8 (confidence: 0.99)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Control Number   1566 non-null   float64
 1   Consumer's Name  1575 non-null   object 
 2   Address          1575 non-null   object 
 3   Previous         1469 non-null   object 
 4   Present          650 non-null    object 
 5   Cons.            835 non-null    float64
 6   Amount           639 non-null    float64
dtypes: float64(3), object(4)
memory usage: 86.3+ KB
None
APR_2020.csv
AUG_2020.csv
FEB_2020.csv
JAN_2020.csv
JUL_2020.csv
JUN_2020.csv
MAR_2020.csv
MAR_APR_2020.csv
MAY_2020.csv
NOV_2020.csv
OCT_2020.csv
SEP_2020.csv
Detected Encoding: utf-8 (confidence: 0.99)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ----

In [ ]:
# Final dataset view in staging column
import utilities as utils
from pathlib import Path

base_dir = Path(f'../../dataset/raw/')
years = ['2024','2025']
source_folder = 'staging'

for year in years:
    inner_path = base_dir / year / source_folder
    files = [f for f in inner_path.iterdir() if f.is_file() and not(f.name in ('staging', 'compiled') or f.name.startswith('COM') or f.name.startswith('STA'))]

    for file in files:
        print(file.name)
        file_path = base_dir / year / source_folder / file.name
        df = utils.read_csv(file_path)
        print(df.info())

APR_2024.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Control Number   2129 non-null   object
 1   Consumer's Name  2129 non-null   object
 2   Address          2129 non-null   object
 3   Previous         2081 non-null   object
 4   Present          2082 non-null   object
 5   Cons.            2065 non-null   object
 6   Amount           1892 non-null   object
dtypes: object(7)
memory usage: 116.6+ KB
None
AUG_2024.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2139 entries, 0 to 2138
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Control No.           684 non-null    float64
 1   Account Name          684 non-null    object 
 2   Service Address       684 non-null    object 
 3   Type                  682 non-null    object 
 4 

# Dataset Final Preparations for Months of July 2024 to May 2025
Should be run manually

| Column | Description |
|--------|-------------|
| `df['0']` | Control numbers |
| `df['2']` | Account names |
| `df['3']` | Service addresses |
| `df['4']` | Types |
| `df['5']` | Meter serial numbers |
| `df['6']` | Previous readings |
| `df['7']` | Present readings |
| `df['8']` | Last present readings |
| `df['9']` | Consumptions |
| `df['13']` | Statuses |

In [25]:
# Final Preparations for compilation
from pathlib import Path 
import pandas as pd 
from utilities import read_csv, save_csv

source_folder = 'staging' 
output_folder = 'compiled' 
years = ['2024', '2025']

raw_col_map = {
    'Control No.': 'Control Number', 
    'Consumer\'s Name': 'Account Name', 
    'Address': 'Service Address', 
    'Type': 'Type', 
    'Previous': 'Previous Reading', 
    'Present': 'Present Reading', 
    'Cons.': 'Consumption'
}

pos_col_map = {
    '0': 'Control No.', 
    '2': 'Account Name', 
    '3': 'Service Address', 
    '4': 'Type', 
    '6': 'Previous Reading', 
    '7': 'Present Reading', 
    '9': 'Consumption', 
    '13': 'Status'
}

for year in years: 
    base_dir = Path(f'../../dataset/raw/{year}/{source_folder}')
    files = [f for f in base_dir.iterdir() if f.is_file() and not(f.name in ('staging', 'compiled') or f.name.startswith('COM') or f.name.startswith('STA'))]

    for file in files:
        df = read_csv(file)
        final_df = pd.DataFrame()

        if any(col_name in df.columns for col_name in ['0', '2', '3', '4', '5', '6', '7', '8', '9', '13']):
            temp_df = df[['0', '2', '3', '4', '5', '6', '7', '8', '9', '13']].copy()
            temp_df.dropna(how='all', inplace=True, axis=0)

            columns = df.columns.str.strip()
            temp_df.rename(columns=pos_col_map, inplace=True)
            new_df = df[['Control No.', 'Account Name', 'Service Address', 'Type', 'Previous Reading', 'Present Reading', 'Consumption', 'Status']].copy()
            temp_df = temp_df[['Control No.', 'Account Name', 'Service Address', 'Type', 'Previous Reading', 'Present Reading', 'Consumption', 'Status']].copy()
            
            final_df = pd.concat([new_df, temp_df], axis=0, ignore_index=True)

        else:
            final_df = df.copy()

        final_df.rename(columns=raw_col_map, inplace=True)
        final_df.dropna(how='all', inplace=True, axis=0)

        subset = ['Account Name', 'Control Number', 'Service Address']
        duplicated = final_df[final_df.duplicated(subset=subset, keep=False)].copy()

        if len(duplicated) > 0:
            dropped_rows = final_df.duplicated(subset=subset, keep='first')
            final_df = final_df.drop_duplicates(subset=subset, keep='first').copy()

            print(f"Dropped {len(dropped_rows)} rows")

        print('Final dataset length:', len(final_df))

        save_csv(final_df, f'../../dataset/raw/{year}/{output_folder}/{file.name}')
        print(f"✅ Saved to {file.name}\n")

print("✅ All files compiled successfully!")


Dropped 2129 rows
Final dataset length: 2128
✅ Saved to APR_2024.csv

Dropped 2139 rows
Final dataset length: 2136
✅ Saved to AUG_2024.csv

Dropped 2159 rows
Final dataset length: 2151
✅ Saved to DEC_2024.csv

Dropped 2112 rows
Final dataset length: 2111
✅ Saved to FEB_2024.csv

Dropped 2115 rows
Final dataset length: 2114
✅ Saved to JAN_2024.csv

Final dataset length: 2122
✅ Saved to JUL_2024.csv

Dropped 2133 rows
Final dataset length: 2130
✅ Saved to JUN_2024.csv

Dropped 2113 rows
Final dataset length: 2112
✅ Saved to MAR_2024.csv

Dropped 2131 rows
Final dataset length: 2130
✅ Saved to MAY_2024.csv

Dropped 2160 rows
Final dataset length: 2152
✅ Saved to NOV_2024.csv

Dropped 2153 rows
Final dataset length: 2147
✅ Saved to OCT_2024.csv

Dropped 2145 rows
Final dataset length: 2139
✅ Saved to SEP_2024.csv

Dropped 2164 rows
Final dataset length: 2163
✅ Saved to APR_2025.csv

Final dataset length: 2159
✅ Saved to FEB_2025.csv

Final dataset length: 2156
✅ Saved to JAN_2025.csv

Drop

# Used for setting standard directory for compiled csv files

In [82]:
base_dir = Path('../../dataset/raw/2020/')
compiled_dir = base_dir / 'compiled'
os.makedirs(compiled_dir, exist_ok=True)

In [83]:
csv_files = [f for f in os.listdir(base_dir) if f.endswith('.csv')]
print(csv_files)

for file in csv_files:
    os.replace(base_dir / file, base_dir / 'compiled' / file)

['APR2020.csv', 'AUG2020.csv', 'DEC2020.csv', 'FEB2020.csv', 'JAN2020.csv', 'JUL2020.csv', 'JUN2020.csv', 'MAR2020.csv', 'MAR_APR2020.csv', 'MAY2020.csv', 'NOV2020.csv', 'OCT2020.csv', 'SEP2020.csv']


In [31]:
import re
from pathlib import Path

year = '2023'
base_dir = Path(f'../../dataset/raw/{year}/compiled/')
files = [f for f in base_dir.iterdir() if f.is_file()]
files.sort()

for f in files:
    orig_name = f.name
    new_name = re.sub(r'(\d)', r'_\1', orig_name, count=1)
    if orig_name != new_name:
        new_path = base_dir / new_name
        f.rename(new_path)
        print(f"Renamed: {orig_name} → {new_name}")
    else:
        print(f"Skipped (no digit): {orig_name}")

Renamed: APR2023.csv → APR_2023.csv
Renamed: AUG2023.csv → AUG_2023.csv
Renamed: DEC2023.csv → DEC_2023.csv
Renamed: FEB2023.csv → FEB_2023.csv
Renamed: JAN2023.csv → JAN_2023.csv
Renamed: JUL2023.csv → JUL_2023.csv
Renamed: JUN2023.csv → JUN_2023.csv
Renamed: MAR2023.csv → MAR_2023.csv
Renamed: MAY2023.csv → MAY_2023.csv
Renamed: OCT2023.csv → OCT_2023.csv
Renamed: SEP2023.csv → SEP_2023.csv
